In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from typing import List
import h5py
from torch.utils.data import Dataset, DataLoader
import numpy as np
import preprocessing
from tqdm import tqdm

from data_utils import preprocess_data, H5Dataset

In [2]:
train_dir = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train/"
train_dir_preprocess = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/"
# train_dir_preprocess = "/clusterfs/ml4hep/mfong/delphes_train_preprocessed/"
train_filepaths = [train_dir + x for x in os.listdir(train_dir)]

In [3]:
preprocess_data(train_filepaths, train_dir_preprocess, force=False)
train_preprocess_file_names = os.listdir(train_dir_preprocess)
train_preprocess_filepaths = [train_dir_preprocess + name for name in train_preprocess_file_names]

100%|██████████| 15/15 [00:00<00:00, 34952.53it/s]

Starting preprocessing on /global/ml4hep/spss/mfong/transfer_learning/delphes_train/train_10.h5
/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_10.h5 is already in target_dir, skipping this file
Starting preprocessing on /global/ml4hep/spss/mfong/transfer_learning/delphes_train/train_5.h5
/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_5.h5 is already in target_dir, skipping this file
Starting preprocessing on /global/ml4hep/spss/mfong/transfer_learning/delphes_train/train_0.h5
/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_0.h5 is already in target_dir, skipping this file
Starting preprocessing on /global/ml4hep/spss/mfong/transfer_learning/delphes_train/train_8.h5
/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_8.h5 is already in target_dir, skipping this file
Starting preprocessing on /global/ml4hep/spss/mfong/transfer_learning/

In [4]:
dataset = H5Dataset(train_preprocess_filepaths, transform=None)

In [5]:
dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

In [6]:
len(dataset)

71742220

In [104]:
x, y = next(iter(dataloader))

In [105]:
# for x, y in dataloader:
#     print(x.shape)
#     print(y.shape)

In [106]:
x.shape

torch.Size([256, 200, 7])

In [107]:
y.shape

torch.Size([256])